Useful imports

In [1]:
import heapq
import bisect
import collections
import itertools
import functools
import math

Assign cookies: https://leetcode.com/problems/assign-cookies/
Video Link: https://youtu.be/DIX2p7vb9co?si=-lYoEBkNX6G40Tpo

In [2]:
# https://leetcode.com/problems/assign-cookies/submissions/1280845661
def findContentChildren(g: list[int], s: list[int]) -> int:
    """
    For each child we assign cookies that only require as much to satisfy their greed.
    For a child with greed of 3, we assign the minimum cookie left to satisfy condition: s[k] >= 3
    """
    # Sort in ASC order since we need to find the max
    # number of content children possible
    g.sort()
    s.sort()

    # Have pointers for greed and cookies arrary
    g_length, s_length = len(g), len(s)
    content_children = g_idx = s_idx = 0

    # Iterate until either of the arrays are exhausted
    while g_idx < g_length and s_idx < s_length:
        if s[s_idx] >= g[g_idx]:
            content_children, s_idx, g_idx = content_children + 1, s_idx + 1, g_idx + 1
        else:
            s_idx += 1

    return content_children

# Testing the solution
assert findContentChildren([1,2,3], [1,1,1]) == 1
assert findContentChildren([1,2], [1,2,3]) == 2
assert findContentChildren([1,2,3], [1,2]) == 2

Shortest job first CPU scheduling
Video Link: https://youtu.be/3-QbX1iDbXs?si=oh3KUu3zLd08uDZ7

In [3]:
def SJF(bt: list[int]) -> float:
    """Time: O(N), Space: O(N)"""

    # Heapify to convert into a min heap
    heapq.heapify(bt)

    N = len(bt)
    curr_time = total_time = 0
    while bt:
        runtime = heapq.heappop(bt)
        total_time += curr_time
        curr_time += runtime

    avg_wait = total_time // N
    return avg_wait

# Testing the solution
assert SJF([4,3,7,1,2]) == 4
assert SJF([1,2,3,4]) == 2

Lemonade change: https://leetcode.com/problems/lemonade-change/description/
Video link: https://youtu.be/n_tmibEhO6Q?si=bqyD-xTjBPIG9scO

In [4]:
# https://leetcode.com/problems/lemonade-change/submissions/1281069969
def lemonadeChange(bills: list[int]) -> bool:
    change5 = change10 = 0
    for bill in bills:
        if bill == 5:
            change5 += 1
        elif bill == 10:
            change10 += 1
            change5 -= 1
        else:
            # Try to change with 10 if available
            if change10 > 0:
                change5, change10 = change5 - 1, change10 - 1
            # Else we can change with 5
            else:
                change5 -= 3

        # Reason this is added here and not at the end: [10,5]
        if change5 < 0 or change10 < 0:
            return False

    else:
        return True

# Testing the solution
assert lemonadeChange([5,5,10,10,20]) == False
assert lemonadeChange([10,5]) == False
assert lemonadeChange([5,10,20]) == False
assert lemonadeChange([5,5,5,20]) == True

Jump game: https://leetcode.com/problems/jump-game/
Video Link: https://youtu.be/tZAa_jJ3SwQ?si=PUeiia1qBXmvfk_C

In [5]:
def canJumpBrute(nums: list[int]) -> bool:
    N = len(nums)
    good_indices: collections.deque = collections.deque([N - 1])
    for i in range(N - 2, -1, -1):
        for good_idx in good_indices:
            if i + nums[i] >= good_idx:
                good_indices.appendleft(i)
                break

    return good_indices[0] == 0

# Testing the solution
assert canJumpBrute([2,3,1,1,4]) == True
assert canJumpBrute([3,2,1,0,4]) == False
assert canJumpBrute([2,0,2,0,0]) == True

In [6]:
# https://leetcode.com/problems/jump-game/submissions/1281092917
def canJump(nums: list[int]) -> bool:
    """
    We can notice that if the latest good index was reachable, then every other good index beyond would
    have been reachable as well.

    On the contrary if the latest good index (nearest to current) was not reachable, then no index beyond would be reachable.

    Time: O(N), Space: O(1)
    """
    N = len(nums)
    good_idx: int = N - 1
    for i in range(N - 2, -1, -1):
        if i + nums[i] >= good_idx:
            good_idx = i

    return good_idx == 0

# Testing the solution
assert canJump([2,3,1,1,4]) == True
assert canJump([3,2,1,0,4]) == False
assert canJump([2,0,2,0,0]) == True

In [7]:
# https://leetcode.com/problems/jump-game/submissions/1281113037
def canJumpStriver(nums: list[int]) -> bool:
    """
    If there were no 0's we would always be able to reach the destination.
    In other cases it may or may not be possible.

    Time: O(N), Space: O(1)
    """
    N, max_idx = len(nums), 0
    for i in range(N):
        if i <= max_idx:
            max_idx = max(max_idx, nums[i] + i)
        else:
            return False

    return True

# Testing the solution
assert canJumpStriver([2,3,1,1,4]) == True
assert canJumpStriver([3,2,1,0,4]) == False
assert canJumpStriver([2,0,2,0,0]) == True

Jump Game - II: https://leetcode.com/problems/jump-game-ii/
Video Link: https://youtu.be/7SBVnw7GSTk?si=UDZojaM70TuG0XgY

In [8]:
def jump2Recursive(nums: list[int]) -> int:
    @functools.cache
    def backtrack(i: int) -> int:
        if i == N - 1:
            return 0
        else:
            min_ = N
            for jump in range(nums[i]):
                min_ = min(min_, backtrack(i + jump + 1))
            return 1 + min_

    N = len(nums)
    return backtrack(0)

# Testing the solution
assert jump2Recursive([2,3,1,1,4]) == 2
assert jump2Recursive([2,3,0,1,4]) == 2

In [9]:
# https://leetcode.com/problems/jump-game-ii/submissions/1281812619/
def jump2Tab(nums: list[int]) -> int:
    # Time: O(N ** 2), Space: O(N)
    N = len(nums)
    dp: list[int] = [N if i < N - 1 else 0 for i in range(N)]
    for i in range(N - 2, -1, -1):
        for j in range(i + 1, min(N, i + 1 + nums[i])):
            dp[i] = min(dp[i], 1 + dp[j])

    return dp[0]

# Testing the solution
assert jump2Tab([2,3,1,1,4]) == 2
assert jump2Tab([2,3,0,1,4]) == 2

In [10]:
# https://leetcode.com/problems/jump-game-ii/submissions/1281839417
def jump2Striver(nums: list[int]) -> int:
    """
    Instead of iterating through all indices that we can jump to,
    we keep track of the range.
    For each range
        - Figure out the maximum idx we can jump to
        - jumps = jumps + 1
    Iterate until we reach the last index, ans would be the value of jumps variable

    Time: O(N), Space: O(1)
    """
    N = len(nums)
    jumps = l = r = 0
    while r < N - 1:
        farthest = 0
        for i in range(l, r + 1):
            farthest = max(farthest, i + nums[i])
        l, r, jumps = r + 1, farthest, jumps + 1

    return jumps

# Testing the solution
assert jump2Striver([2,3,1,1,4]) == 2
assert jump2Striver([2,3,0,1,4]) == 2

Job sequencing Problem: https://www.geeksforgeeks.org/problems/job-sequencing-problem-1587115620/1
Video Link: https://youtu.be/QbwltemZbRg?si=R93V-44Jyr37Il4L

In [11]:
def jobScheduling(jobs: list[tuple[int, int, int]], N: int) -> tuple[int, int]:
    """
    Start with the jobs with highest profit, try to push the start time as close as possible to the deadline
    being super lazy about it.

    Time: O(N log N) + O(N x max deadline)
    Space: O(N)

    We can optimize the inner loop (deadline -> 0) further with the use of DSU (Disjoint set union)
    """
    jobs_start: set[int] = set()
    jobs_sorted: list[tuple[int, int]] = sorted(map(lambda x: (x[2], x[1]), jobs), reverse=True)
    job_count = total_profit = 0
    for profit, deadline in jobs_sorted:
        for start in range(deadline - 1, -1, -1):
            if start not in jobs_start:
                jobs_start.add(start)
                job_count, total_profit = job_count + 1, total_profit + profit
                break

    return job_count, total_profit

# Testing the solution
assert jobScheduling([(1,2,100),(2,1,19),(3,2,27),(4,1,25),(5,1,15)], 5) == (2, 127)
assert jobScheduling([(1,4,20),(2,1,10),(3,1,40),(4,1,30)], 5) == (2, 60)
assert jobScheduling([(1,4,50),(2,1,50)], 2) == (2, 100)
assert jobScheduling([(1,4,60),(2,1,50)], 2) == (2, 110)

N meetings in one room: https://www.geeksforgeeks.org/problems/n-meetings-in-one-room-1587115620/1
Video Link: https://youtu.be/mKfhTotEguk?si=gTVT-RsSg0PR9RPP

In [12]:
def maximumMeetings(N: int, start: list[int], end: list[int]) -> int:
    """
    Sort based on end time, we need to find meetings that finish sooner.
    Simply iterate through the sorted meeting in order and check if it is possible to host the meeting

    Time: O(N log N)
    Space: O(N)
    """

    # Sort meeting based on the end time
    meetings: list[tuple[int, int, int]] = [(start[i], end[i], i) for i in range(N)]
    meetings.sort(key=lambda x: x[1])

    order: list[int] = []
    meeting_count = free_at = 0
    for stime, etime, idx in meetings:
        if stime > free_at:
            order.append(idx)
            meeting_count, free_at = meeting_count + 1, etime

    # If needed, to return the order in which the meetings could be conducted,
    # we can make use of the `meeting_count` variable

    return meeting_count

# Testing the solution
assert maximumMeetings(6, [1,3,0,5,8,5], [2,4,6,7,9,9]) == 4
assert maximumMeetings(3, [1,4,6], [5,6,10]) == 2